In [ ]:
def create_cartesian_grid(latitude, longitude, dx, extra_pixels=7):
    """ 
    Creates a cartesian grid (regular in distance, kilometers) from a geodesic latitude, longitude grid. 
    The new grid is expressed in latitude, longitude coordinates.

    Parameters
    ----------
    longitude : numpy ndarray 
        Vector of longitude for geodesic input grid. 
    latitude : numpy ndarray 
        Vector of latitude for geodesic input grid. 
    dx : float 
        Grid spacing in kilometers. 
    extra_pixels : int, optional
        Number of extra pixels to add on each side of the grid. Default is 2.

    Returns
    -------
    ENSLAT2D : 
        2-D numpy ndarray of the latitudes of the points of the cartesian grid 
    ENSLON2D : 
        2-D numpy ndarray of the longitudes of the points of the cartesian grid 
    """
    km2deg = 1 / 111

    # Extend the latitude range by extra_pixels grid points on each side
    ENSLAT = np.arange(latitude[0] - extra_pixels * dx * km2deg, latitude[-1] + (extra_pixels + 1) * dx * km2deg, dx * km2deg)
    range_lon = longitude[-1] - longitude[0]
    
    # Extend by the number of extra pixels on each side
    if longitude.size % 2 == 0:
        nstep_lon = floor(range_lon / (dx * km2deg)) + 2 * (extra_pixels + 1)  
    else:
        nstep_lon = ceil(range_lon / (dx * km2deg)) + 2 * (extra_pixels + 1) 

    ENSLAT2D = np.repeat(np.expand_dims(ENSLAT, axis=1), axis=1, repeats=nstep_lon)

    # ENSEMBLE OF LONGITUDES
    mid_lon = (longitude[-1] + longitude[0]) / 2
    ENSLON2D = np.zeros_like(ENSLAT2D)

    for i in range(len(ENSLAT)):
        d_lon = dx * km2deg * (np.cos(np.pi * latitude[0] / 180) / np.cos(np.pi * ENSLAT[i] / 180))
        d_lon_range = np.array([j * d_lon for j in range(1, int(nstep_lon / 2) + 1)])
        lon_left = np.flip(mid_lon - d_lon_range)
        lon_right = mid_lon + d_lon_range
        ENSLON2D[i, :] = np.concatenate((lon_left, [mid_lon], lon_right))[:nstep_lon]

    return ENSLAT2D, ENSLON2D, ENSLAT2D.shape[0], ENSLAT2D.shape[1]


In [ ]:
def lowpass(_lambda,nx,ny,wavenum2D) : 
    _lowpass = np.zeros((3*ny,3*nx))
    for i in range (3*ny):
        for j in range(3*nx):
            if wavenum2D[i,j]<1/_lambda:
                _lowpass[i,j] = 1
    return _lowpass 

In [ ]:
def extend(ssh,nx,ny):
    ssh_extended = np.empty((3*ny,3*nx))
    ssh_extended[ny:2*ny,nx:2*nx] = +ssh
    ssh_extended[0:ny,nx:2*nx] = +ssh[::-1,:]
    ssh_extended[2*ny:3*ny,nx:2*nx] = +ssh[::-1,:]
    ssh_extended[:,0:nx] = ssh_extended[:,nx:2*nx][:,::-1]
    ssh_extended[:,2*nx:3*nx] = ssh_extended[:,nx:2*nx][:,::-1]
    return ssh_extended

In [ ]:
def gaspari_cohn(array,distance,center):
    """
    NAME 
        bfn_gaspari_cohn

    DESCRIPTION 
        Gaspari-Cohn function. @vbellemin.
        
        Args: 
            array : array of value whose the Gaspari-Cohn function will be applied
            center : centered value of the function 
            distance : Distance above which the return values are zeros


        Returns:  smoothed values 
            
    """ 
    if type(array) is float or type(array) is int:
        array = np.array([array])
    else:
        array = array
    if distance<=0:
        return np.zeros_like(array)
    else:
        array = 2*np.abs(array-center*np.ones_like(array))/distance
        gp = np.zeros_like(array)
        i= np.where(array<=1.)[0]
        gp[i]=-0.25*array[i]**5+0.5*array[i]**4+0.625*array[i]**3-5./3.*array[i]**2+1.
        i =np.where((array>1.)*(array<=2.))[0]
        gp[i] = 1./12.*array[i]**5-0.5*array[i]**4+0.625*array[i]**3+5./3.*array[i]**2-5.*array[i]+4.-2./3./array[i]
        #if type(r) is float:
        #    gp = gp[0]
    return gp

In [ ]:
def create_spatial_window(nx,ny):
    result = np.ones((3*ny,3*nx))
    
    gaspari_x = np.expand_dims(gaspari_cohn(np.arange(2*nx),nx,nx),axis=0)
    gaspari_y = np.expand_dims(gaspari_cohn(np.arange(2*ny),ny,ny),axis=1)

    #paving edges with gaspari-cohn
    result[2*ny:,nx:2*nx] = np.repeat(gaspari_y[ny:,:],repeats=nx,axis=1)
    result[:ny,nx:2*nx] = np.repeat(gaspari_y[:ny,:],repeats=nx,axis=1)
    result[ny:2*ny,0:nx] = np.repeat(gaspari_x[:,:nx],repeats=ny,axis=0)
    result[ny:2*ny,2*nx:] = np.repeat(gaspari_x[:,nx:],repeats=ny,axis=0)

    #paving corners with gaspari-cohn
    result[2*ny:,2*nx:]=gaspari_y[ny:,:]*gaspari_x[:,nx:]
    result[:ny,:nx]=gaspari_y[:ny,:]*gaspari_x[:,:nx]
    result[2*ny:,:nx]=gaspari_y[ny:,:]*gaspari_x[:,:nx]
    result[:ny,2*nx:]=gaspari_y[:ny,:]*gaspari_x[:,nx:]

    return result 

In [ ]:
def extract_bar_tide(ssh0,dx):

    nx = ssh0.shape[1]
    ny = ssh0.shape[0]

    kx = np.fft.fftfreq(3*nx,dx) # km
    ky = np.fft.fftfreq(3*ny,dx) # km
    k, l = np.meshgrid(kx,ky)
    wavenum2D = np.sqrt(k**2 + l**2)

    lambda_bar = 400
    lowpass_bar = lowpass(lambda_bar,nx,ny,wavenum2D)

    window = create_spatial_window(nx,ny)

    ssh = extend(ssh0,nx,ny)
    ssh = ssh * window
    ssh_freq = fp.fft2(ssh)
    ssh_freq_filtered = lowpass_bar * ssh_freq
    ssh_filtered = np.real(fp.ifft2(ssh_freq_filtered))[ny:2*ny,nx:2*nx]

    return ssh_filtered


In [ ]:
def doble_plot(ssh_bar, ssh_igw):
    """
    Plots ssh_bar and ssh_igw side by side with colorbars labeled as 'Height (m)'.
    
    Parameters:
    - ssh_bar: xarray DataArray containing the barotropic tide data
    - ssh_igw: xarray DataArray containing the internal gravity waves data
    """
    
    # Create a figure and two subplots
    fig, axs = plt.subplots(1, 2)

    # Plot ssh_bar
    im1 = axs[0].pcolormesh(ssh_bar.lon, ssh_bar.lat, ssh_bar, shading='auto',cmap=cmocean.cm.curl)
    divider1 = make_axes_locatable(axs[0])
    cax1 = divider1.append_axes("right", size="5%", pad=0.05)
    cb1 = fig.colorbar(im1, cax=cax1)
    # cb1.set_label('Height (m)')
    axs[0].set_title(r'$\eta^{BAR}$')
    # axs[0].set_xlabel('Longitude')
    # axs[0].set_ylabel('Latitude')
    axs[0].set_aspect('equal', 'box')

    # Plot ssh_igw
    im2 = axs[1].pcolormesh(ssh_igw.lon, ssh_igw.lat, ssh_igw, shading='auto',cmap=cmocean.cm.curl)
    divider2 = make_axes_locatable(axs[1])
    cax2 = divider2.append_axes("right", size="5%", pad=0.05)
    cb2 = fig.colorbar(im2, cax=cax2)
    cb2.set_label('Height (m)')
    axs[1].set_title(r'$\eta^{IGW}$')
    # axs[1].set_xlabel('Longitude')
    # axs[1].set_ylabel('Latitude')
    axs[1].set_aspect('equal', 'box')

    # Adjust layout
    plt.tight_layout()
    plt.show()

In [ ]:
def apply_mask(ds, mask):
    ssh_hf = ds.ssh_hf.where(mask == 1, np.nan)
    return ssh_hf.coarsen(x=4, y=4, boundary='trim').mean()

def fill_missing_data(ssh_hf):
    x_axis = Axis(ssh_hf.x.values, is_circle=True)
    y_axis = Axis(ssh_hf.y.values, is_circle=True)
    grid = Grid2D(y_axis, x_axis, ssh_hf.values.transpose(0, 1))
    has_converged, filled = fill.gauss_seidel(grid)
    ssh_hf_filled = ssh_hf.copy(deep=True, data=filled)
    return ssh_hf_filled

def create_cartesian_grid_data(ssh_hf_filled, dx):
    ENSLAT2D, ENSLON2D, i_lat, i_lon = create_cartesian_grid(ssh_hf_filled.lat[:, 0].values,
                                                             ssh_hf_filled.lon[0, :].values,
                                                             dx)
    return ENSLAT2D, ENSLON2D, i_lat, i_lon

def interpolate_to_cartesian(ssh_hf_filled, ENSLAT2D, ENSLON2D):
    points = np.column_stack((ssh_hf_filled.lat.data.flatten(), ssh_hf_filled.lon.data.flatten()))
    values = ssh_hf_filled.data.flatten()
    target_grid = np.column_stack((ENSLAT2D.flatten(), ENSLON2D.flatten()))

    array_cart_ssh = griddata(points, values, target_grid, method='linear')
    return array_cart_ssh.reshape(ENSLAT2D.shape)

def fill_cartesian_nans(array_cart_ssh, i_lat, i_lon):
    x_axis = Axis(np.arange(i_lon))
    y_axis = Axis(np.arange(i_lat))
    
    grid = Grid2D(y_axis, x_axis, array_cart_ssh)
    has_converged, filled = fill.gauss_seidel(grid)
    
    return filled

def extract_barotropic_tide(cart_ssh_hf, dx):
    return np.array(extract_bar_tide(cart_ssh_hf.values, dx))

def interpolate_back_to_original_grid(cart_ssh_filtered, ENSLAT2D, ENSLON2D, ssh_hf):
    lon2d, lat2d = np.meshgrid(ssh_hf.lon[0, :].values, ssh_hf.lat[:, 0].values)
    
    ssh_bar_array = np.array([griddata(
        np.array([ENSLAT2D.flatten(), ENSLON2D.flatten()]).T,
        cart_ssh_filtered.flatten(),
        (lat2d, lon2d),
        method='linear'
    )])
    
    ssh_bar = xr.DataArray(
        data=ssh_bar_array[0],
        dims=ssh_hf.dims,
        coords=ssh_hf.coords,
        name='ssh_bar'  # Set the name here
    )
    
    return ssh_bar

def process_data_for_time_step(time_step, ds, mask, dx):
    ds_time = ds.sel(time_counter=time_step)
    ssh_bar, ssh_igw = process_data(ds_time, mask, dx)
    return ssh_bar, ssh_igw

def process_data(ds, mask, dx):
    ssh_hf = apply_mask(ds, mask)
    
    ssh_hf_filled = fill_missing_data(ssh_hf)
    ENSLAT2D, ENSLON2D, i_lat, i_lon = create_cartesian_grid_data(ssh_hf_filled, dx)
    
    array_cart_ssh = interpolate_to_cartesian(ssh_hf_filled, ENSLAT2D, ENSLON2D)
    filled = fill_cartesian_nans(array_cart_ssh, i_lat, i_lon)
    
    cart_ssh_hf = xr.DataArray(data=filled,
                               dims=["y", "x"],
                               coords=dict(y=(["y"], np.arange(i_lat)), x=(["x"], np.arange(i_lon))))
    
    cart_ssh_filtered = extract_barotropic_tide(cart_ssh_hf, dx)
    
    ssh_bar = interpolate_back_to_original_grid(cart_ssh_filtered, ENSLAT2D, ENSLON2D, ssh_hf)
    
    ssh_igw = ssh_hf_filled - ssh_bar
    
    # Set the name for ssh_igw
    ssh_igw.name = 'ssh_igw'
    
    return ssh_bar, ssh_igw



In [ ]:
# def interpolate_to_target_grid(source_data, target_lat, target_lon, method='linear'):
#     """
#     Interpolates source_data to the target latitude and longitude grid using the specified method.
#     """
#     source_lat = source_data.lat.values
#     source_lon = source_data.lon.values
#     source_values = source_data.values

#     # Interpolating using griddata with the specified method
#     interpolated_data = griddata(
#         np.array([source_lat.flatten(), source_lon.flatten()]).T,
#         source_values.flatten(),
#         (target_lat, target_lon),
#         method=method
#     )
    
#     return interpolated_data